In [5]:
import pandas as pd
import json
import re
import geopy.distance
import requests
from lxml import etree

DATA = "C:/Users/fabien/Desktop/OpenDataToulouse/Fraicheur/Data"

In [11]:
###### arbre-d-alignement-reformated ######

df = pd.read_csv(DATA + "/arbres-d-alignement.csv", sep=";")

treeType = set()
for index, row in df.iterrows():
    
    #Compute length of the way
    parseGeoShape = json.loads(df.loc[index,"Geo Shape"])
    wayLength = 0
    if parseGeoShape["type"] == "LineString":
        points = parseGeoShape["coordinates"]
        for i in range(1, len(points)):
            wayLength += geopy.distance.distance(tuple(points[i-1]), tuple(points[i])).m
    elif parseGeoShape["type"] == "MultiLineString":
        section = parseGeoShape["coordinates"]
        for points in section:
            for i in range(1, len(points)):
                wayLength += geopy.distance.distance(tuple(points[i-1]), tuple(points[i])).m
    df.loc[index, "Longueur de la voie"] = wayLength
    
    #Retrieve number of trees for each type of trees in Patrimoine and compute the total number of trees
    parsePatrimoine = re.findall("([0-9]+)\s([A-Z-\s]+)", row["patrimoine"])
    nbrTrees = 0
    for couple in parsePatrimoine:
        df.loc[index, couple[1].rstrip()] = int(couple[0])
        nbrTrees += int(couple[0])
    df.loc[index, "Nombre total d'arbre"] = nbrTrees
    

df = df.fillna(0).drop(["Geo Point", "Geo Shape", "adresse", "patrimoine"], axis=1)
df.to_csv(DATA + "/arbres-d-alignement-reformated.csv", index=False, sep=";")
df

,ID,Longueur de la voie,FRENE,Nombre total d'arbre,CHENE VERT,MICOCOULIER,ERABLE,PLATANE,LAGERSTROEMIA,PEUPLIER,...,CAIGLUS,PIN NOIRS,CEDRE ATLAS,ACACIAS,ERABLE MONTPELLIER,CYPRES CHAUVE,PAULOWNIA,MERISIER A GRAPPES,ERABLE DE ROTTERDAM,LAGAERSTROEMIA
0,523,263.971149,21.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,544,172.907937,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,517,96.347291,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,296,551.835785,0.0,37.0,0.0,30.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,73,257.923941,0.0,28.0,0.0,0.0,0.0,28.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,84,448.355086,0.0,93.0,0.0,0.0,0.0,93.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,115,155.648432,0.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,117,593.920734,0.0,44.0,0.0,37.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,171,157.024571,0.0,10.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,172,68.383343,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
###### fountain ######

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:25];
area["name"="Toulouse"]["admin_level"="8"]->.boundaryarea;
way(area.boundaryarea)["amenity"="fountain"];
  foreach(
    node(w)->.d;
    .n is_in->.a;
    area.a[name][boundary=administrative][admin_level~"^[2-8]$"] -> .a;
    out center;
    convert way ::=::,
              ::id = id(),
              is_in=a.set("{" + t["admin_level"] + ":" + t["name"] + "}");

    out;
  );
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

fountainData = {"lat": [], "lon": [], "name": [], "id":[]}
for element in data["elements"]:
    if "is_in" not in element["tags"]:
        fountainData["id"].append(element["id"])
        fountainData["lat"].append(element["center"]["lat"])
        fountainData["lon"].append(element["center"]["lon"])
        if "name" in element["tags"]:
            fountainData["name"].append(element["tags"]["name"])
        else:
            fountainData["name"].append("Inconnu")
fountain = pd.DataFrame(fountainData)
fountain.to_csv(DATA + "/fontaine.csv", sep=";", encoding="ISO-8859-1", index=False)
fountain

,id,lat,lon,name
0,74708358,43.599202,1.381279,Inconnu
1,197920877,43.595079,1.408507,Inconnu
2,613543303,43.585106,1.428310,Inconnu
3,615749199,43.587889,1.436621,Inconnu
4,614098975,43.598257,1.434585,Inconnu
5,206565273,43.591074,1.453166,Inconnu
6,22545000,43.595762,1.452586,Inconnu
7,182599631,43.593229,1.451739,Inconnu
8,594654009,43.597184,1.451064,Inconnu
9,295737122,43.602046,1.444540,Fontaine des Puits-Clos


In [13]:
###### OSM_ways_to_kml ######

def OSM_ways_to_kml(data):
    nodes = {}
    ways = {}
    for element in data["elements"]:
        ID = str(element["id"])
        if element["type"]=="node":
            nodes[ID] = [str(element["lon"]), str(element["lat"])]
        if element["type"]=="way":
            ways[ID] = {"nodes": element["nodes"], "name": "Inconnu"}
            if "tags" in element:
                if "name" in  element["tags"]:
                    ways[ID]["name"] = element["tags"]["name"]

    kml = etree.Element("kml")
    kml.attrib["xmlns"] = "http://www.opengis.net/kml/2.2"
    document = etree.SubElement(kml, "Document")
    for key in ways.keys():
        way = etree.SubElement(document, "Placemark")
        name = etree.SubElement(way, "name")
        name.text = key
        description = etree.SubElement(way, "description")
        description.text = ways[key]["name"]
        polygon = etree.SubElement(way, "Polygon")
        outerBoundaryIs = etree.SubElement(polygon, "outerBoundaryIs")
        LinearRing = etree.SubElement(outerBoundaryIs, "LinearRing")
        coordinates = etree.SubElement(LinearRing, "coordinates")
        coordinates.text = "\n".join([",".join(nodes[str(ID)]) for ID in ways[key]["nodes"]])
        
    return etree.tostring(kml, encoding="UTF-8", pretty_print=True, xml_declaration=True)

with open(DATA + '/park_osm.txt') as json_file:
    data = json.load(json_file)

kml = OSM_ways_to_kml(data)

with open(DATA + '/park_osm.kml', 'wb') as f:
    f.write(kml)

kml

b'<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<kml xmlns="http://www.opengis.net/kml/2.2">\n  <Document>\n    <Placemark>\n      <name>4546118</name>\n      <description>Parc de la Grande Plaine</description>\n      <Polygon>\n        <outerBoundaryIs>\n          <LinearRing>\n            <coordinates>1.4922143,43.5936354\n1.4922468,43.5934015\n1.4938895,43.5877971\n1.4917499,43.5874308\n1.491146,43.589388\n1.4908798,43.5902505\n1.4909556,43.5903907\n1.4909937,43.5904923\n1.4908925,43.5906369\n1.4907211,43.5908521\n1.490576,43.5913262\n1.4904912,43.591617\n1.4905398,43.5919287\n1.4907788,43.5925588\n1.4909443,43.5929296\n1.4910767,43.5930826\n1.4912079,43.5932109\n1.4913928,43.5933853\n1.4915977,43.5935457\n1.4916874,43.5936078\n1.4917618,43.5936354\n1.4922143,43.5936354</coordinates>\n          </LinearRing>\n        </outerBoundaryIs>\n      </Polygon>\n    </Placemark>\n    <Placemark>\n      <name>4722589</name>\n      <description>Inconnu</description>\n      <Polygon>\n        <o

In [26]:
###### find_path_for_nearest_destination ######

def find_path_for_nearest_destination(departures, destinations, dist_tresh=2, api_key="b5b9132c-704e-480f-a3ff-2b5dd45d4113"):
    nearestDestinationPath = {}
    test=0
    for departureKey in departures.keys():
        minTime = 5*3600*1000
        nearestDestinationPath[departureKey] = {"id": None}
        for destinationKey in destinations.keys():
            if geopy.distance.distance(tuple(departures[departureKey]), tuple(destinations[destinationKey])).km < dist_tresh:

                info = { 
                "profile": "foot",
                "start": departures[departureKey],
                "end": destinations[destinationKey],
                "key": api_key
                }

                graphHopper_request = "https://graphhopper.com/api/1/route?" + \
                "&point=" + str(info["start"][0]) + "," + str(info["start"][1]) +  \
                "&point=" + str(info["end"][0]) + "," + str(info["end"][1]) + \
                "&vehicle=" + info["profile"] + \
                "&points_encoded=false" + \
                "&key=" + info["key"]

                response = requests.get(graphHopper_request)
                temp = json.loads(response.text)

                print("Request for")
                print((departureKey, destinationKey))
                print("Request status : " + str(response.status_code))
                print("Travel time: " + str(temp["paths"][0]["time"]) + " ms")
                print("Actual min time: " + str(minTime) + " ms")

                way = {}
                way["id"] = destinationKey
                way["time"] = temp["paths"][0]["time"]
                way["distance"] = temp["paths"][0]["distance"]
                way["path"] = temp["paths"][0]["points"]["coordinates"]

                if minTime > way["time"]:
                    nearestDestinationPath[departureKey] = way
                    minTime = way["time"]

        if nearestDestinationPath[departureKey]["id"] is None:
            print(departureKey + " too far from locations")
            
    return nearestDestinationPath

In [29]:
df = pd.read_csv(DATA + "/recensement-population-2015-grands-quartiers-activite.csv", encoding='latin-1', sep=";")
districts = {}
for index, row in df.iterrows():
    districts[row["Libelle des grands quartiers"]] = row["Geo Point"].split(",")

df = pd.read_csv(DATA + "/fontaine.csv", sep=";", encoding="ISO-8859-1")
fountains = {}
for index, row in df.iterrows():
    fountains[row["id"]] = [row["lat"], row["lon"]]
    
nearestFountain = find_path_for_nearest_destination(districts, fountains)

with open('filename.pickle', 'wb') as handle:
    pickle.dump(DATA + "nearestFountain.pickle", handle, protocol=pickle.HIGHEST_PROTOCOL)

ARNAUD BERNARD too far from locations
COTE PAVEE too far from locations
LES PRADETTES too far from locations
SAINT-SIMON too far from locations
Request for
('LE BUSCA', 206565273)
Request status : 200
Travel time: 71379 ms
Actual min time: 18000000 ms
GUILHEMERY too far from locations
PONT-DES-DEMOISELLES too far from locations
MONTAUDRAN - LESPINET too far from locations
PATTE D'OIE too far from locations
ROSERAIE too far from locations
PAPUS too far from locations
EMPALOT too far from locations
Request for
('SAINT-CYPRIEN', 614098975)
Request status : 200
Travel time: 15998 ms
Actual min time: 18000000 ms
SOUPETARD too far from locations
CARMES too far from locations
COMPANS too far from locations
MINIMES too far from locations
LA TERRASSE too far from locations
Request for
('LES CHALETS', 628608217)
Request status : 200
Travel time: 260226 ms
Actual min time: 18000000 ms
FAOURETTE too far from locations
CASSELARDIT too far from locations
LALANDE too far from locations
CHATEAU-DE-L'H

{'AMIDONNIERS': {'id': None},
 'ARENES': {'id': None},
 'ARNAUD BERNARD': {'id': None},
 'BAGATELLE': {'id': None},
 'BARRIERE-DE-PARIS': {'id': None},
 'BASSO-CAMBO': {'id': None},
 'BELLEFONTAINE': {'id': None},
 'BONNEFOY': {'id': None},
 'CAPITOLE': {'id': None},
 'CARMES': {'id': None},
 'CASSELARDIT': {'id': None},
 "CHATEAU-DE-L'HERS": {'id': None},
 'COMPANS': {'id': None},
 'COTE PAVEE': {'id': None},
 'CROIX-DAURADE': {'id': None},
 'CROIX-DE-PIERRE': {'id': None},
 'EMPALOT': {'id': None},
 'FAOURETTE': {'id': None},
 'FER-A-CHEVAL': {'id': None},
 'FONTAINE-LESTANG': {'id': None},
 'GINESTOUS': {'id': None},
 'GRAMONT': {'id': None},
 'GUILHEMERY': {'id': None},
 'JULES JULIEN': {'id': None},
 'JUNCASSE - ARGOULETS': {'id': None},
 'LA CEPIERE': {'id': None},
 'LA FOURGUETTE': {'id': None},
 'LA TERRASSE': {'id': None},
 'LALANDE': {'id': None},
 'LARDENNE': {'id': None},
 'LE BUSCA': {'distance': 99.141,
  'id': 206565273,
  'path': [[1.453888, 43.590495],
   [1.453594, 43

In [ ]:
df = pd.read_csv(DATA + "/recensement-population-2015-grands-quartiers-activite.csv", encoding='latin-1', sep=";")
districts = {}
for index, row in df.iterrows():
    districts[row["Libelle des grands quartiers"]] = row["Geo Point"].split(",")

df = pd.read_csv(DATA + "/fontaine.csv", sep=";", encoding="ISO-8859-1")
fountains = {}
for index, row in df.iterrows():
    fountains[row["id"]] = [row["lat"], row["lon"]]
    
nearestFountain = find_path_for_nearest_destination(districts, fountains)

with open('filename.pickle', 'wb') as handle:
    pickle.dump(DATA + "nearestFountain.pickle", handle, protocol=pickle.HIGHEST_PROTOCOL)